In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io as io

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, normaltest
from sklearn.preprocessing import OneHotEncoder

# **1. Wybór tematu i zbioru danych**

Do realizacji projektu wykorzystano bazę danych zawierającą **Oceny tabliczek czekolady** w zależności od danych czynników. Analizę danych wykonano przy użyciu języka Python i wsparciu bibliotek zaimportowanych powyżej. 

**Opis zbioru danych:**

* W omawianej bazie danych znajdują się następujące dane:
* Company – nazwa firmy wytwarzającej czekoladę.
* Specific Bean Origin or Bar Name – specyficzny region geograficzny, z którego pochodzi czekolada. 
* REF – wartość powiązana z momentem, gdy ocena trafiła do bazy danych. Im wyższa tym nowsza.
* Review Date – data publikacji recenzji.
* Cocoa Percent – procentowy udział kakao w tabliczce czekolady.
* Company Location – kraj producenta. 
* Rating – ocena eksperta dla danej tabliczki czekolady. 
* Bean Type – odmiana użytego ziarna, jeżeli została dostarczona.
* Broad Bean Origin – geo-region pochodzenia ziarna.

**Cel Analizy**

Celem analizy jest sprawdzenie jakie czynniki mają wpływ na wybór najlepszej tabliczki czekolady przez ekspertów.

W badaniu dokonam analizy jaki wpływ na ostateczny werdykt ekspertów mają takie zmienne jak zawartość kakao, kraju pochodzenia ziarna czy kraj producenta.
 
Każda z hipotez opiera się o inną zmienną zależną i jaki wpływ ma dana zmienna na ocenę czekolady przez recenzenta. 

**Hipotezy**

Do zbioru danych można postawić 3 hipotezy, w oparciu o 3 różne zmienne zależne.

* Czekolady z niską zawartością kakao są lepiej oceniane przez recenzentów. 

* Pochodzenie ziaren kakaowca ma kluczowy wpływ na walory smakowe czekolady.

* Lokalizacja firm wpływa na finalną jakość produkowanej czekolady. 

# **2. Czyszczenie i analiza danych**

Na początku wczytano bazę danych z pliku do zmiennej. 
Następnie zmodyfikowano nazwy kolumn w pobranych danych, aby były bardziej czytelne i nie zawierały niepotrzebnych znaków specjalnych.
Przygotowano również kopie danych, tak aby mieć zachowany pierwotny kształt danych oraz aby w wykresach analizy dany bazować na oryginale. Dodatkowo w tabeli oryginału, dane procentowe zamieniono na wartości dziesiętne, aby w łatwy sposób przedstawić dane procentowe 

Dane przedstawiono w postaci tabeli, gdzie wyświetlono pierwsze 5 rekordów.

In [ ]:
#Wczytanie bazy danych do lokalnej zmiennej
data = pd.read_csv("../input/chocolate-bar-ratings/flavors_of_cacao.csv")
data.columns = ['Company', 'Origin', 'REF', 'Review_Date', 'Cocoa_Percent', 'Company_Location','Rating', 'Bean_Type', 'Bean_Org']
originalData = data.copy()
originalData['Cocoa_Percent']=(originalData['Cocoa_Percent']).str.replace('%', ' ')
originalData['Cocoa_Percent']=(originalData['Cocoa_Percent']).astype(float)
data.head()

Aby dane były przedstawione w bardziej "elegancki" sposób, wszystkie kolumny z typem danych Object zamieniono na typ Category. Pozwoli nam to bardziej efektywniej tworzyć wizualizację danych niż w postaci typu Object.

In [ ]:
data['Company'] = data['Company'].astype('category')
data['Origin'] = data['Origin'].astype('category')
data['Company_Location'] = data['Company_Location'].astype('category')
data['Bean_Type'] = data['Bean_Type'].astype('category')
data['Bean_Org'] = data['Bean_Org'].astype('category')
data.info()

Przed rozpoczęcie pracy nad bazą danych, należy przebadać czy w modelu nie występują jakieś braki. W przypadku tej bazy danych występowały braki danych w tabeli "Bean_Type" oraz "Bean_Org". Do pozbycia się braków użyłem funkcji dropna() z parametrem inplace=True.

In [ ]:
data.dropna(subset = ["Bean_Type"], inplace=True)
data.dropna(subset = ["Bean_Org"], inplace=True)
data.isna().sum()

Stworzono funkcję pomocniczą do zamiany wartości procentowej na wartość float z przedziału 0-1.

In [ ]:
def p2f(x):
    return float(x.strip('%'))/100
data['Cocoa_Percent'] = data['Cocoa_Percent'].str.rstrip('%').astype('float') / 100.0

Wyświetlenie danych po zmianie z procentów na wartość dziesiętną.

In [ ]:
data.head()

Opis zmiennych ilościowych przy użyciu funkcji describe. 

In [ ]:
data.describe()

**Opracuj podstawowe statystyki dla każdej zmiennej ilościowej ze swoich hipotez.**

Zmiennymi ilościowymi jest zawartość kakao, ocena czekolady przez ekspertów, wartość z informacją, kiedy został dodany rekord oraz rok publikacji recenzji.

In [ ]:
qty_col = ['Cocoa_Percent', 'REF', 'Review_Date','Rating']
qty_stat = data[qty_col].agg(["count","mean","median","min", "max", "std", "var",])
qty_stat = qty_stat.append(data[qty_col].mode().rename(index={0:"mode"}))
qty_stat

Opis zmiennych jakościowych o typie Category.

In [ ]:
data.describe(include="category")

**Opracuj tabele liczności dla każdej zmiennej jakościowej z hipotez**

Zmiennymi jakościowymi będą kolumny, które nie zawierają wartości liczbowych. Dla tej bazy danych będą to kolumny - Company, Origin, Bean Type, Bean Org.

In [ ]:
quali_cols = ['Company', 'Origin', 'Company_Location','Bean_Type', 'Bean_Org']
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres rozkładu liczności dla {col}")
    sns.histplot(data[col], ax=axes[i])
    print(f"""Dla kolumny {col} tabela liczności wygląda następująco:\n\n{pd.DataFrame(data[col].value_counts())}\n""")

Na powyższych histogramach można zauważyć, że dostarczone dane nie posiadają zbliżonego rozkładu prawdopodobieństwa. Każdy z tych wykresów ma jakąś wartość najczestszą. Szczegółowa analiza tych danych odbędzie się w dalszej części sprawozdania.

**Podsumowanie wszystkich pomiarów** 

In [ ]:
def showSummaryOfMeasurements():
    print("Podsumowanie wszystkich pomiarow")
    print(data.describe(include='all').T)

In [ ]:
showSummaryOfMeasurements()

* Z przedstawionych danych widzimy, że średnia zawartość kakao jest bliska 70%. 
* Średnia z wszystkich ocen ekspertów wacha się w granicach 3,19. 
* Tylko dwie czekolady z wszystkich 1795 danych została oceniona na maksymalną ocenę równą 5 i tylko cztery na ocenę równą 
* Najwięcej recenzji pochodzi z roku 2012, podczas gdy początkowe zbieranie danych rozpoczęło się w 2006r. i było sporządzane do 2017 kiedy dane zostały udostępnione. 
* Największym producentem ziaren kakaowca jest zdecydowanie Wenezuela. 
* Ciekawostka: 2 najlepiej ocenione czekolady zostały ocenione w 2006 i 2007 i to one są odnośnikiem dla kolejnych zapisywanych do bazy danych.

**Macierz korelacji zmiennych ilościowych.**

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(data.corr(), annot=True, ax=ax)
data.corr()

Z przedstawionej macierzy korelacji wynika, że najbardziej skorelowane są zmienne Review Date i REF. To się zgadza, gdyż im większa wartość REF, tym późniejsza jest data recenzji czekolady. Mimo wszystko ten przypadek możemy smiało wykluczyć, gdyż dotyczy on zmiennych, które nie będą wchodzić w skład analizy i tak na prawdę ta korelacja nie ma większego znaczenia.

Widzimy również dwie korelację, które warto odnotować:
* Korelacja ujemna między zmienną zawartości kakao a oceną czekolady (-0.16). Można to tłumaczyć tak, że wraz spadkiem zawartości kakao, ocena tabliczki czekolady wzrasta. Korelacja jest dość słaba, ale jest widoczna. 
* Druga korelacja dotyczy zależności między datą dodania oceny a zwiększoną oceną. (0.1) Tę korelację możemy odrzucić, ponieważ data publikacji oceny nie może mieć znaczenia na ocene danej tabliczki czekolady. Nie jest to czynnik, który może realnie wpływać na smak czekolady. Nie można logicznie wytłumaczyć takiej korelacji.

**Analiza własna na podstawie sporządzonych wykresów.**

In [ ]:
def showRatingDistribution():
    sns.countplot(x='Rating', data=data)
    plt.xlabel('Ocena')
    plt.ylabel('Liczba ekspertow')
    plt.title('Rozklad ocen pomiedzy ekspertow')
    print('Rozklad ocenionych tabliczek czekolady')

In [ ]:
showRatingDistribution()

Najważniejszą kwestią, którą porusza wykres jest informacja jak dużo z czekolad zostało już ocenionych.

Eksperci najczęściej oceniają produkty jako zadowalające, a najczęściej wybieraną wartością jest 3.5, czyli lekko powyżej średniej. Wartości niskie i maksymalne są unikane przez ekspertów. Świadczy to o tym, że serwowane czekolady są „z góry” na pewnym dobrym poziomie i tylko pewne parametry świadczą o ich niskiej ocenie. 

In [ ]:
def showPercentOfCocoa():
    sns.displot(data=originalData['Cocoa_Percent']) 
    plt.xlabel('Zawartosc procentowa kakao')
    plt.ylabel('Ilosc')
    plt.title('Ilosc procentowa kakao w tabliczkach czekolady')
    print('Ilosc tabliczek po zawartosci kakao')


In [ ]:
showPercentOfCocoa()

Badania przeprowadzone wykazują, że największą częścią z tabliczek czekolady stanowią te o zawartości kakao równą 70%. 

60% i mniej zawartości kakao obecne jest w mniej niż 100 rodzajach czekolady. Ich ilość jest nawet mniejsza od produktów o zawartości większej niż 80%. 

Produkty pomiędzy 70 a 80% zawartością kakao stanowią fundament dla czekolad, a stosunek ich ilości w bazie danych  stanowi prawie 75%.

Porównując zawartość kakao do ocen ekspertów, wynika, że najlepiej oceniane są najpopularniejsze tabliczki zawierające 60-80% kakao, z czego dominującą cześć stanowią 70%.

Ciekawą obserwacją danych jest fakt, że pomiędzy nimi nie dochodzi do żadnych widocznych korelacji ani wzoru w jakim zachowują się dane. Dzieje się to z uwagi na bardzo duże rozproszenie danych. Wynika z tego, że preferencje recenzentów są w dużej mierze zmienne. Wyjątkiem są wartości dla niskiej zawartości kakao (o poziomach 42% i 46%), gdzie oceny są jednakowe, jednak mała ilość danych może tłumaczyć to zjawisko. 

In [ ]:
def showSupplierCountries():
    data['Company_Location'].value_counts().head(15)
    data['Company_Location'].value_counts().head(15).plot(kind='barh')
    plt.xlabel('Liczba czekolad')
    plt.ylabel('Kraje dystrubucji')
    print("15 krajow z najwieksza dostawa czekolad")

In [ ]:
showSupplierCountries()

Kolejnym parametrem uwzględnionym w bazie danych są kraje, które mają najwyższych sprzedawców czekolady. 

Dane przedstawiają 15 największych krajów, które realizują dostawę czekolad. Z wykresu jasno widzimy, że Stany Zjednoczone są światowym producentem tabliczek czekolady. Odpowiadają za 764 produktów z całej bazy danych.

Te informacje mogą pomóc nam określić jakie przychody mają poszczególne firmy w danych krajach oraz na tej podstawie określić czy warto inwestować w dystrybucje produktów w danym kraju. Duża ilość firm pozwala nam też określić potencjalną liczbę klientów, która musi być większa niż w pozostałych krajach, w których udział eksportu jest niższy. 

In [ ]:
def showRatingOf100Cocoa():
    cocoa_one_hundred=originalData[originalData['Cocoa_Percent'] == 100.0] 
    cocoa_one_hundred.count()
    sns.countplot(x='Rating', data=cocoa_one_hundred, color='red')
    print("Czekolady z 100% zawartoscia kakao")

In [ ]:
showRatingOf100Cocoa()

**Czekolady 100% kakao:**
Z wykresu przedstawiającego czekolady o 70% zawartości kakao jest bardzo ciekawy. Mimo, że dysponujemy tylko 19 danymi to oceny są bardzo od siebie rozbieżne. Widzimy też, że żadna z tych czekolad nie uzyskała wyniku powyżej 3,5 co jest wynikiem mocno poniżej normy. Dodatkowo wszystkie pochodziły z różnych części świata i są sprzedawane w różnych firmach. 

Niskie wyniki mogą wynikać z wysokiej ceny, jest to gatunek premium, smak – czekolady o takiej zawartości kakao są z pewności bardzo gorzkie oraz to, że te czekolady zawierają samo kakao, bez możliwych dodatków poprawiających smak produktu.

In [ ]:
def showRatingOf70Cocoa():
    cocoa_seventy=originalData[originalData['Cocoa_Percent' ] == 70.0]
    cocoa_seventy.count()
    sns.countplot(x='Rating', data=cocoa_seventy, color='pink')
    print("Czekolady z 70% zawartoscia kakao")

In [ ]:
showRatingOf70Cocoa()

**Czekolady 70% kakao:**
Jest to zdecydowanie najczęściej stosowana proporcja kakao w czekoladzie, ich ilość jest plasowana na 672. Sam wykres informuje nas, że taka zawartość ma największe walory smakowe. Tylko kilka osiąga wartość poniżej pewnej normy. Rodzaj ziarna w połączeniu z zawartością kakao powoduje, że można uzyskać zarówno czekolady o ocenach poniżej 2, jak i równej 5. 
Jednak całe zestawienie daje jasno do zrozumienia, że nie bez przyczyny jest to najczęściej wykorzystywany wariant do produkcji, gdyż po prostu z uwagi na idealne proporcje jest na te czekolady największy popyt wśród konsumentów na całym świecie.

Taka analiza pokazuje też, że wartość w macierzy korelacji miała swoje odzwierciedlenie i była prawidłowa. 

**Test normalny dla zmiennych - oznacz wartości odstające**

Sporzadzono wykresy rozkładu wartości zmiennych ilościowych, aby zweryfikować czy dana analiza jest potrzebna i przydatna do analizy. 

In [ ]:
fig, axes = plt.subplots(len(qty_col), 1, figsize=(10,30))
for i, col in enumerate(qty_col):
    axes[i].set_title(f"Wykres rozkładu liczności dla {col}")
    sns.histplot(data[col], ax=axes[i], kde=True)

In [ ]:
for i, col in enumerate(qty_col):
    print(f"Wartość p dla testu normalnego dla kolumny {col} wynosi {normaltest(data[col].values).pvalue}\n")

Dla żadnej z tych zmiennych nie ma rozkładu gaussa. Z tego powodu nie możemy zastosować parametrycznych metod statystycznych w celu przewidywania zmiennych zależnych.

**Wykonaj wykresy ramka-wąsy dla wszystkich zmiennych ilościowych z hipotez. Wybierz dwie pary zmiennych (ilościowa-jakościowa) i wykonaj wykresy.**

In [ ]:
fig, axes = plt.subplots(len(qty_col),1,  figsize=(10,30))
for i, col in enumerate(qty_col):
    axes[i].set_title(f"Wykres pudełkowy dla {col}")
    sns.boxplot(data = data, x=data[col], ax=axes[i], orient = "h")

Wykresy ramka-wąsy pozwalają zapoznać się z rozkładem zmiennych w wybranym zespole danych. Dzięki nim można łatwo zauważyć wartości, które odstają od reszty w postaci punktów wystających poza wąsy. 

# **3. Indukcja drzew decyzyjnych.**

Przypomnę jakie hipotezy zostały sformuowane na wstępie:

**Czekolady z niską zawartością kakao są lepiej oceniane przez recenzentów.**

**Pochodzenie ziaren kakaowca ma kluczowy wpływ na walory smakowe czekolady.**

**Lokalizacja firm wpływa na finalną jakość produkowanej czekolady.**

Do sprawdzenia pierwszej hipotezy wykorzystano zatem zmienną: zawartość kakao oraz ocena ekspertów, która będzie użyta w każdej z hipotez. Do hipotez użyto po jednej zmiennej niezależne, po to aby utworzyć 3 różne hipotezy. Baza nie jest duża i ciężko utworzyć połaczenia między różnymi zmiennymi dla tej bazy danych.

In [ ]:
# biblioteki
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, precision_recall_fscore_support
pd.options.mode.chained_assignment = None

# oddzielenie danych
df = pd.read_csv("../input/chocolate-bar-ratings/flavors_of_cacao.csv")

df['Cocoa\nPercent'] = df['Cocoa\nPercent'].str.replace('%', '')
df['Cocoa\nPercent'] = df['Cocoa\nPercent'].str.replace('.', '')
df['Cocoa\nPercent'] = df['Cocoa\nPercent'].astype(int)

def normalizeIt(percent):
    if percent > 100:
        percent = int(str(percent)[:2])
    return percent

df['Cocoa\nPercent'] = df['Cocoa\nPercent'].apply(normalizeIt)

df['Rating'] = (df['Rating']* 100).astype(int)
df['Rating'].head(5)

hipo_1_data = df[["Cocoa\nPercent"]]
hipo_1_y = df["Rating"]


X_train, X_test, y_train, y_test = train_test_split(hipo_1_data, hipo_1_y, train_size=0.7, random_state=1)
X_train

tree_model = DecisionTreeClassifier(min_samples_leaf=200)

tree_model.fit(X_train, y_train)

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

Dokładności modelu wynosi 22.63%

Potwierdzenie ważności predyktorów. Dla moich drzew będzie to zawsze 100% udział jednej zmiennej. 

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_1_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_1_data.shape[1]), hipo_1_data.columns)
plt.ylim([-1, hipo_1_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

**Wygenerowane drzewo:**

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns)
plt.show()

Do hipotezy nr. 2 wykorzystano zmienną jakościową pochodzenia ziarna. Niestety w tym przypadku nie udało się finalnie wygenerować drzewa decyzjnego, gdyż występuje błąd z wartościami NaN. Zdiagnozowałem, że jest to przede wszystkich spowodowane tym, że w bazie danych nie mamy stricte pustych wartości, tylko wartości z odstępem w komórce. Do tego dochodzi również dekodowanie miasto pochodzenia ziarna na wartość liczbową, ale nie wszystkie wartości zostały zmapowane, gdyż unikalnych wartości wykryło w tym przypadku 100. Niestety nie starczyło mi już czasu na poprawę tych problemów, ale zrobiłbym wypisanie wartościom unikalnym kolejne iterowane numery. 

Zakomentowano wartość, aby nie pojawiał się błąd w raporcie. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, precision_recall_fscore_support
pd.options.mode.chained_assignment = None

# oddzielenie danych

hipo_2_data = data[["Bean_Org"]]
hipo_2_y = data["Rating"]

decode = {
    "Sao Tome":1,"Togo":2,"Peru":3,"Venezuela":4,"Cuba":5,"Panama":6,"Madagascar":7,"Brazil":8,"Ecuador":9,"Colombia":10,"Burma":11,"Papua New Guinea":12,
    "Bolivia":13,"Fiji":14,"Mexico":15,"Indonesia":16,"Trinidad":17,"Vietnam":18,"Nicaragua":19,"Tanzania":20,"Dominican Republic":21,"Ghana":22,"Belize":23,
    "Jamaica":24,"Grenada":25,"Guatemala":26,"Honduras":27,"Costa Rica":28,"Haiti":29,"Congo":30,"Philippines":31,"Malaysia":32,"Dominican Rep., Bali":33,
    "Cameroon":34,"Venez,Africa,Brasil,Peru,Mex":35,"Gabon":36,"Ivory Coast":37,"Sri Lanka":38,"Puerto Rico":39,"Uganda":40,"Martinique":41,"Sao Tome & Principe":42,
    "Vanuatu":43,"Australia":44,"Ecuador, Costa Rica":45,"West Africa":46,"Honduras":47,"St. Lucia":48,"Cost Rica, Ven":49,"Peru, Madagascar":50,"Trinidad, Tobago":51,
    "Ven, Trinidad, Ecuador":52,"Africa, Carribean, C. Am.":53,"Ven., Indonesia, Ecuad.":54,"Trinidad-Tobago":55,"Peru, Ecuador, Venezuela":56,"Liberia":57,
    "Venezuela, Dom. Rep.":58,"Hawaii":59,"Colombia, Ecuador":60,"Solomon Islands":61,"Peru, Belize":62,"Peru, Mad., Dom. Rep.":63,"PNG, Vanuatu, Mad":64,
    "El Salvador":65,"South America":66,"Ghana, Domin. Rep":67,"Venezuela, Java":68,"Venezuela/ Ghana":69,"Venezuela, Ghana":70,"Indonesia, Ghana":71,
    "Peru(SMartin,Pangoa,nacional)":72,"Principe":73,"Carribean":74,"Central and S. America":75,"Ven., Trinidad, Mad.":76,"Carribean(DR/Jam/Tri)":77,
    "Ghana & Madagascar":78,"Ven.,Ecu.,Peru,Nic.":79,"Madagascar & Ecuador":80,"Guat., D.R., Peru, Mad., PNG":81,"Peru, Dom. Rep":82,"Dom. Rep., Madagascar":83,
    "Gre., PNG, Haw., Haiti, Mad":84,"Mad., Java, PNG":85,"Ven, Bolivia, D.R.":86,"DR, Ecuador, Peru":87,"Suriname":88,"Ecuador, Mad., PNG":89,"Ghana, Panama, Ecuador":90,
    "Venezuela, Carribean":91,"Central and S. America":92
}

hipo_2_data.loc[:, "Bean_Org"] = hipo_2_data.Bean_Org.map(decode).values

X_train, X_test, y_train, y_test = train_test_split(hipo_2_data, hipo_2_y, train_size=0.7, random_state=1)
X_train

tree_model = DecisionTreeClassifier(min_samples_leaf=200)

# tree_model.fit(X_train, y_train)

# preds = tree_model.predict(X_test)
# acc = accuracy_score(y_test, preds)
# print(f"Dokładność modelu wynosi {100*acc:.2f}%")

In [ ]:
# var_importances = tree_model.feature_importances_
# std = np.std(var_importances,axis=0)
# indices = np.argsort(var_importances)
# plt.figure()
# plt.title("Ważność predyktorów")
# plt.barh(range(hipo_2_data.shape[1]), var_importances[indices],
#        color="r", xerr=std, align="center")
# plt.yticks(range(hipo_2_data.shape[1]), hipo_2_data.columns)
# plt.ylim([-1, hipo_2_data.shape[1]])
# plt.grid(b=True)
# plt.xlim(0, 1)
# plt.show()

Trzecie drzewo decyzyjne zostało wygenerowane. Drzewo zrealizowano w oparciu o zmienną lokalizacja miasta produkcyjnego, jaki wpływ ma na rating.

In [ ]:
# biblioteki
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, precision_recall_fscore_support
pd.options.mode.chained_assignment = None

# oddzielenie danych
df = pd.read_csv("../input/chocolate-bar-ratings/flavors_of_cacao.csv")
df.columns = ['Company', 'Origin', 'REF', 'Review_Date', 'Cocoa_Percent', 'Company_Location','Rating', 'Bean_Type', 'Bean_Org']

df['Rating'] = (df['Rating']* 100).astype(int)
df['Rating'].head(5)

hipo_3_data = df[["Company_Location"]]
hipo_3_y = df["Rating"]

decode = {"U.S.A.": 1, "France": 2, "Canada": 3, "U.K.": 4, "Italy": 5, "Ecuador":6, "Australia":7, "Belgium":8, "Switzerland":9, "Germany":10, "Austria":11,
          "Spain":12, "Colombia":13, "Hungary":14, "Venezuela":15, "New Zealand":16, "Madagascar":17, "Peru":18, "Japan":19, "Brazil":20, "Denmark":21, "Vietnam":22,
          "Guatemala":23, "Scotland":24, "Israel":25, "Costa Rica":26, "Argentina":27, "Poland": 28, "Lithuania":29, "Honduras":30, "Sweden":31, "Nicaragua":32,
          "Domincan Republic":33, "South Korea":34, "Sao Tome": 35, "Puerto Rico":36, "Amsterdam":37, "Mexico":38, "Ireland":39, "Netherlands":40, "Fiji":41,
          "South Africa":42, "Portugal":43, "Iceland":44, "Singapore":45, "Grenada":46, "Finland":47, "Bolivia":48, "Chile":49, "St. Lucia":50, "Czech Republic": 51,
          "Eucador":52,"Niacragua":53,"Suriname":54,"Ghana":55, "India":56, "Russia":57, "Martinique":58, "Philippines":59, "Wales":60}

hipo_3_data.loc[:, "Company_Location"] = hipo_3_data.Company_Location.map(decode).values

X_train, X_test, y_train, y_test = train_test_split(hipo_3_data, hipo_3_y, train_size=0.7, random_state=1)
X_train

tree_model = DecisionTreeClassifier(min_samples_leaf=200)

tree_model.fit(X_train, y_train)

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

Ważność predyktorów w 100% do 1 zmiennej. 

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_3_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_3_data.shape[1]), hipo_3_data.columns)
plt.ylim([-1, hipo_3_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

Wygenerowane drzewo decyzyjne dla 3 hipotezy.

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns)
plt.show()

# **4. Analiza skupień.**

Do wykonania analizy skupień wybrano zmienne:

1. Cocoa_Percent
2. Rating
3. Company_Location

Celem analizy jest sprawdzenie, czy możemy wyznaczyć konkretne grupy tabliczek czekoldy w zależności od zawartości kakao, produkowanych w różnych miastach i jaki ma to wpływ na finalną ocenę. 

In [ ]:
from sklearn.cluster import KMeans
pd.options.mode.chained_assignment = None
from sklearn.decomposition import PCA

decode = {"U.S.A.": 1, "France": 2, "Canada": 3, "U.K.": 4, "Italy": 5, "Ecuador":6, "Australia":7, "Belgium":8, "Switzerland":9, "Germany":10, "Austria":11,
          "Spain":12, "Colombia":13, "Hungary":14, "Venezuela":15, "New Zealand":16, "Madagascar":17, "Peru":18, "Japan":19, "Brazil":20, "Denmark":21, "Vietnam":22,
          "Guatemala":23, "Scotland":24, "Israel":25, "Costa Rica":26, "Argentina":27, "Poland": 28, "Lithuania":29, "Honduras":30, "Sweden":31, "Nicaragua":32,
          "Domincan Republic":33, "South Korea":34, "Sao Tome": 35, "Puerto Rico":36, "Amsterdam":37, "Mexico":38, "Ireland":39, "Netherlands":40, "Fiji":41,
          "South Africa":42, "Portugal":43, "Iceland":44, "Singapore":45, "Grenada":46, "Finland":47, "Bolivia":48, "Chile":49, "St. Lucia":50, "Czech Republic": 51,
          "Eucador":52,"Niacragua":53,"Suriname":54,"Ghana":55, "India":56, "Russia":57, "Martinique":58, "Philippines":59, "Wales":60}

cluster_data = originalData[["Cocoa_Percent", "Rating", "Company_Location"]]
cluster_data.loc[:, "Company_Location"] = cluster_data.Company_Location.map(decode).values

sse = []

# ładowanie modelu
for k in range(1,11):
    cluster_model = KMeans(n_clusters=k)
    cluster_model.fit(cluster_data)
    sse.append(cluster_model.inertia_)
    
# sprawdzanie jaką wartość k wybrać - metoda łokcia

plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Wartość k")
plt.ylabel("SSE")
plt.show()

Z powyższego wykresu widzimy, że liczba klastrów, którą powinnyśmy użyć to wartość k=2, ale aby to zweryfikować użyjemy też biblioteki silhouette, która wskaże nam najbardziej optymalną liczbe klastrów, gdybyśmy mieli nieoczywisty przypadek. 

In [ ]:
from sklearn.metrics import silhouette_score

sil_score_max = -1
best_n_clusters = 0

for k in range(2,11):
  cluster_model = KMeans(n_clusters = k)
  labels = cluster_model.fit_predict(cluster_data)
  sil_score = silhouette_score(cluster_data, labels)
  print(f"Średnia wartość dla {k} klastrów wynosi {sil_score}")
  if sil_score > sil_score_max:
    sil_score_max = sil_score
    best_n_clusters = k
    
print(f"Najlepsza ilość klastrów to: {best_n_clusters}")

In [ ]:
cluster_model = KMeans(n_clusters = best_n_clusters)
# trenowanie modelu
result = cluster_model.fit_predict(cluster_data)

labels = cluster_model.labels_

result_data = cluster_data.copy()
result_data["labels"] = labels

results_0 = cluster_data[result_data.labels == 0]
results_1 = cluster_data[result_data.labels == 1]

Analiza klastra pierwszego:

In [ ]:
results_0.describe()

Drugi klaster:

In [ ]:
results_1.describe()

Analiza skupień metodą EM.

In [ ]:
# załadowanie modułu do analizy metodą EM
from sklearn.mixture import GaussianMixture

# wczytanie modelu z dwoma klastrami
em_model = GaussianMixture(n_components=2, random_state=0)

labels = em_model.fit_predict(cluster_data)

#labels = em_model.labels_

result_data = cluster_data.copy()
result_data["labels"] = labels

results_0 = cluster_data[result_data.labels == 0]
results_1 = cluster_data[result_data.labels == 1]

Pierwszy klaster:

In [ ]:
results_0.describe()

Drugi klaster:

In [ ]:
results_1.describe()

Wykresy dystrybucji zmiennych w klastrach:

In [ ]:
import scipy.stats as stats
import math

for col in results_1:
    mu_0 = results_0[col].mean()
    variance_0 = results_0[col].var()
    mu_1 = results_1[col].mean()
    variance_1 = results_1[col].var()
    sigma_0 = math.sqrt(variance_0)
    sigma_1 = math.sqrt(variance_1)
    x_0 = np.linspace(mu_0 - 3*sigma_0, mu_0 + 3*sigma_0, 100)
    x_1 = np.linspace(mu_1 - 3*sigma_1, mu_1 + 3*sigma_1, 100)
    plt.figure(figsize=(8,5))
    plt.title(f"Wykres dystrybucji zmiennych w klastrach dla zmiennej {col}")
    plt.plot(x_0, stats.norm.pdf(x_0, mu_0, sigma_0))
    plt.plot(x_1, stats.norm.pdf(x_1, mu_1, sigma_1))
    plt.show()

Z przeprowadzonej analizy można wywnioskować, że niemożliwe jest wyznaczenie konkretnych grup.

# **5. Wybrany algorytm data mining.**

# Data Mining - Las losowy dla przewidywania ocen. 

Importujemy bibliotekę do trenowania danych oraz klasyfikacji za pomocą lasu losowego.
Standardowo robimy porządki w danych oraz dzielimy dane na testowe i treningowe.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("../input/chocolate-bar-ratings/flavors_of_cacao.csv")

df['Cocoa\nPercent'] = df['Cocoa\nPercent'].str.replace('%', '')
df['Cocoa\nPercent'] = df['Cocoa\nPercent'].str.replace('.', '')
df['Cocoa\nPercent'] = df['Cocoa\nPercent'].astype(int)

def normalizeIt(percent):
    if percent > 100:
        percent = int(str(percent)[:2])
    return percent

df['Cocoa\nPercent'] = df['Cocoa\nPercent'].apply(normalizeIt)

df['Rating'] = (df['Rating']* 100).astype(int)
df['Rating'].head(5)

company = pd.get_dummies(df['Company\xa0\n(Maker-if known)'],drop_first=True)
sbOrigin = pd.get_dummies(df['Specific Bean Origin\nor Bar Name'],drop_first=True)
companyLocation = pd.get_dummies(df['Company\nLocation'],drop_first=True)
bType = pd.get_dummies(df['Bean\nType'],drop_first=True)
bbOrigin = pd.get_dummies(df['Broad Bean\nOrigin'],drop_first=True)

df = pd.concat([df, company, sbOrigin, companyLocation, bType, bbOrigin], axis = 1)

df.drop(['Company\xa0\n(Maker-if known)', 'Specific Bean Origin\nor Bar Name','Company\nLocation', 'Bean\nType', 
         'Broad Bean\nOrigin'], axis = 1, inplace = True )
df = df.loc[:,~df.columns.duplicated()]

X = df.drop('Rating', axis = 1) #Features
y = df['Rating']   # Target Variables
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=7)

rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)

Porównajmy jak te modele się prezentują.

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test,rfc_pred))

In [ ]:
print(accuracy_score(y_test,rfc_pred)*100)

Najwięcej ocen jest pomiędzy 3 a 3.5.

In [ ]:
sns.countplot(x = 'Rating', data=df)

Dlatego pogrupujmy na wartości całkowite.

In [ ]:
def rating_to_int(rating):
    
    rating = int(rating)
    
    if (rating == 0.0 ):
        return 0.0
    elif (rating > 0 ) and (rating <= 199 ):
        return 1.0
    elif (rating >= 200 ) and (rating <= 299 ):
        return 2.0
    elif (rating >= 300 ) and (rating <= 399 ):
        return 3.0
    else:
        return 4.0

Wdrażamy te wartości.

In [ ]:
df['Rating'] = df['Rating'].apply(rating_to_int)

Tak obecnie wygląda to na wykresie:

In [ ]:
sns.countplot(x = 'Rating', data=df)

Najwięcej ocen jest dla wartości 3, dlatego, że najwiecej ocen jest wlasnie pomiędzy 3 a 3.5

In [ ]:
X = df.drop('Rating', axis = 1)
y = df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=7)

In [ ]:
rfc = RandomForestClassifier(n_estimators=5000, min_weight_fraction_leaf= 0)
rfc.fit(X_train, y_train)

In [ ]:
rfc_pred = rfc.predict(X_test)
print(classification_report(y_test,rfc_pred))

In [ ]:
print(accuracy_score(y_test,rfc_pred)*100)